In [1]:
!pip install -q snowflake-connector-python
!pip install -q deltalake

StatementMeta(, 31e141bc-f752-4204-b613-d948a30f23bd, 3, Finished, Available)

**Authentication**

In [2]:
import configparser
config = configparser.ConfigParser()
config.read("/lakehouse/default/Files/snowflake.ini")
snowflakeuser = config.get("myvars", "snowflakeuser")
snowflakepass = config.get("myvars", "snowflakepass")
accountx      = config.get("myvars", "accountx")
warehousex    = config.get("myvars", "warehousex")
databasex     = config.get("myvars", "databasex")
schemax       = config.get("myvars", "schemax")

StatementMeta(, 31e141bc-f752-4204-b613-d948a30f23bd, 4, Finished, Available)

In [3]:
import snowflake.connector
import os
def getconnection():
  ctx = snowflake.connector.connect(
      user=snowflakeuser,
      password=snowflakepass,
      account=accountx,
      warehouse= warehousex,
      database= databasex,
      schema =schemax
      )
  cs = ctx.cursor()
  try:
          cs.execute("ALTER SESSION SET USE_CACHED_RESULT=true")
          
  finally:
          cs.close()
          return ctx

StatementMeta(, 31e141bc-f752-4204-b613-d948a30f23bd, 5, Finished, Available)

In [4]:
conn = getconnection()
cs = conn.cursor()
cs.execute('SELECT * FROM  lineitem limit 10000000')


StatementMeta(, 31e141bc-f752-4204-b613-d948a30f23bd, 6, Finished, Available)

In [7]:
from deltalake.writer import write_deltalake
from trident_token_library_wrapper import PyTridentTokenLibrary
aadToken = PyTridentTokenLibrary.get_access_token("storage")
path = "abfss://data@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/snowflake/raw"

StatementMeta(, 31e141bc-f752-4204-b613-d948a30f23bd, 9, Finished, Available)

In [8]:
write_deltalake(path, cs.fetch_arrow_all(),mode='append',storage_options={"bearer_token": aadToken, "use_fabric_endpoint": "true"})

StatementMeta(, 31e141bc-f752-4204-b613-d948a30f23bd, 10, Finished, Available)

In [9]:
for result in cs.fetch_arrow_batches():
 write_deltalake(path, result,mode='append',storage_options={"bearer_token": aadToken, "use_fabric_endpoint": "true"})

StatementMeta(, 31e141bc-f752-4204-b613-d948a30f23bd, 11, Finished, Available)